<a href="https://colab.research.google.com/github/min-hee-Lee/python_workdemo/blob/master/(TF_IDF)shareit_review_wordcloud_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 워드클라우드와 텍스트마이닝

In [ ]:
#구글 코랩 한글
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/n

In [ ]:
#드라이브연결
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/python/data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/python/data


Jpype설치
- python에서 java라이브러리를 이용할 수 있도록 설치한다.

In [ ]:
#http://www.lfd.uci.edu/~gohlke/pythonlibs/#jpype
#아래 부분 실행하면 설치됨
!pip install -U "jpype1<1.1"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 한글을 사용하기 위해 세종사전을 이용
!pip install konlpy
!pip install wordcloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### KoNLPY(코엔엘파이)
- KoNLPY에서는 대한민국 헌법 말뭉치인 kolaw와 국회법안 말뭉치인 kobill을 제공한다

import

In [ ]:
## 기본
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

#데이터 전처리
from konlpy.tag import Okt
from collections import Counter
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Mecab
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
# 현재 코랩에 설치돼 있는 것 확인
#!pip list

### 데이터 가져오기

In [ ]:
#utf-8 codec can's decode byte 0xb9 in position 0: invalid start byte
df = pd.read_csv('./shareit_review_TF_IDF.csv', encoding='cp949')
gyms = list(df['Gym'])
review = list(df['Review'])
df.head()


,User_name,Gender,Gym,Grade,Review
0,이**,M,명지전문대학 예체능관 실내체육관,5,최저기온 영하15도에도 불구 따뜻하게 즐농했습니다.
1,김**,M,경기대학교(서울) 본관 실내체육관,5,기록적 추위에도 따뜻하고 깨끗한 체육관에서 즐겁게 운동했습니다
2,이**,M,한성대학교 상상관 실내체육관,5,개인적으로 사용한 체육관 시설 중 시설과 장비는 가장 좋았습니다. 다만 정말 상당히...
3,황**,F,N sports 실내체육관,4,체육관 정보에는 28 x 15 라고 나와있는데 막상 코트에서 뛰어보니 28 x 15...
4,박**,F,삼성썬더스 농구클럽 구리다산점 체육관A,5,아주만족했어요


In [ ]:
df.shape

(392, 5)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   User_name  392 non-null    object
 1   Gender     392 non-null    object
 2   Gym        392 non-null    object
 3   Grade      392 non-null    int64 
 4   Review     392 non-null    object
dtypes: int64(1), object(4)
memory usage: 15.4+ KB


In [ ]:
# 각 체육관별 명사를 담은 딕셔너리 만들기

noun_dict = {
  '명지전문대학 예체능관 실내체육관':['바닥','주차','가격','난방','화장실','시설'],
  '경기대학교(서울) 본관 실내체육관': ['바닥','주차','가격','난방','화장실','시설'],
  '한성대학교 상상관 실내체육관': ['바닥','주차','가격','난방','화장실','시설'],
  'N sports 실내체육관' :['바닥','주차','가격','난방','화장실','시설'],
  '삼성썬더스 농구클럽 구리다산점 체육관A' : ['바닥','주차','가격','난방','화장실','시설']
}

print(noun_dict)

{'명지전문대학 예체능관 실내체육관': ['바닥', '주차', '가격', '난방', '화장실', '시설'], '경기대학교(서울) 본관 실내체육관': ['바닥', '주차', '가격', '난방', '화장실', '시설'], '한성대학교 상상관 실내체육관': ['바닥', '주차', '가격', '난방', '화장실', '시설'], 'N sports 실내체육관': ['바닥', '주차', '가격', '난방', '화장실', '시설'], '삼성썬더스 농구클럽 구리다산점 체육관A': ['바닥', '주차', '가격', '난방', '화장실', '시설']}


In [ ]:
from konlpy.tag import Komoran
from collections import defaultdict

In [ ]:
# Komoran 객체 생성
komoran = Komoran()

# 호텔 별 명사 빈도수를 저장할 딕셔너리 생성
gym_nouns_dict = {}

for gym in gyms:
  gym_nouns_dict[gym] = defaultdict(int)

  
# 댓글에서 명사 추출하여 빈도수 계산
for i in range(len(review)):
    nouns = komoran.nouns(review[i])
    gym = gyms[i]
    for noun in nouns:
        if noun in noun_dict.get(gym, []):
          gym_nouns_dict[gym][noun] += 1
        

# for gym in gyms:
#     new_df = df.loc[df.Gym==gym,['Review']]
#     reviewList = list(new_df['Review'])
#     for i in range(len(reviewList)):
#          nouns = komoran.nouns(reviewList[i])
#          for noun in nouns:
#               if noun in noun_dict.get(gym, []):
#                 gym_nouns_dict[gym][noun] += 1



#결과 출력
for gym in gym_nouns_dict:
    print(gym)
    
    #print(gym_nouns_dict[gym])
    for k in gym_nouns_dict[gym]:
       print(k)
       print( gym_nouns_dict[gym][k])
       break
   

   

명지전문대학 예체능관 실내체육관
바닥
7
경기대학교(서울) 본관 실내체육관
주차
7
한성대학교 상상관 실내체육관
시설
22
N sports 실내체육관
바닥
12
삼성썬더스 농구클럽 구리다산점 체육관A
주차
5


In [ ]:
gyms = list(df['Gym'])
gyms

['명지전문대학 예체능관 실내체육관',
 '경기대학교(서울) 본관 실내체육관',
 '한성대학교 상상관 실내체육관',
 'N sports 실내체육관',
 '삼성썬더스 농구클럽 구리다산점 체육관A',
 '한성대학교 상상관 실내체육관',
 'N sports 실내체육관',
 '명지전문대학 예체능관 실내체육관',
 '한성대학교 상상관 실내체육관',
 '경기대학교(서울) 본관 실내체육관',
 '삼성썬더스 농구클럽 구리다산점 체육관A',
 '경기대학교(서울) 본관 실내체육관',
 '삼성썬더스 농구클럽 구리다산점 체육관A',
 '한성대학교 상상관 실내체육관',
 '한성대학교 상상관 실내체육관',
 '삼성썬더스 농구클럽 구리다산점 체육관A',
 '명지전문대학 예체능관 실내체육관',
 'N sports 실내체육관',
 '경기대학교(서울) 본관 실내체육관',
 '삼성썬더스 농구클럽 구리다산점 체육관A',
 '경기대학교(서울) 본관 실내체육관',
 'N sports 실내체육관',
 '한성대학교 상상관 실내체육관',
 '삼성썬더스 농구클럽 구리다산점 체육관A',
 '명지전문대학 예체능관 실내체육관',
 '삼성썬더스 농구클럽 구리다산점 체육관A',
 '경기대학교(서울) 본관 실내체육관',
 '삼성썬더스 농구클럽 구리다산점 체육관A',
 '한성대학교 상상관 실내체육관',
 '경기대학교(서울) 본관 실내체육관',
 '삼성썬더스 농구클럽 구리다산점 체육관A',
 '한성대학교 상상관 실내체육관',
 '한성대학교 상상관 실내체육관',
 '경기대학교(서울) 본관 실내체육관',
 'N sports 실내체육관',
 '명지전문대학 예체능관 실내체육관',
 '삼성썬더스 농구클럽 구리다산점 체육관A',
 '한성대학교 상상관 실내체육관',
 '경기대학교(서울) 본관 실내체육관',
 '삼성썬더스 농구클럽 구리다산점 체육관A',
 'N sports 실내체육관',
 '경기대학교(서울) 본관 실내체육관',
 '삼성썬더스 농구클럽 구리다산점 체육관A',
 'N sports 실내체육관',
 '경기대학교(서울) 본관 

In [ ]:
data_list = list()
gym_list=list()
for gym in gym_nouns_dict:
    print(gym)
    gym_list.append(gym)
    data_str=list()
    #print(gym_nouns_dict[gym])
    for k in gym_nouns_dict[gym]:
       print(k)
      
       #print(gym_nouns_dict[gym][k])
       
       for  i in range(gym_nouns_dict[gym][k]):
         
          data_str.append( "".join(k))
    data_list.append(data_str)
print(data_list )


명지전문대학 예체능관 실내체육관
바닥
가격
주차
난방
시설
화장실
경기대학교(서울) 본관 실내체육관
주차
가격
시설
바닥
난방
화장실
한성대학교 상상관 실내체육관
시설
바닥
난방
주차
가격
화장실
N sports 실내체육관
바닥
주차
가격
시설
화장실
난방
삼성썬더스 농구클럽 구리다산점 체육관A
주차
바닥
난방
시설
화장실
가격
[['바닥', '바닥', '바닥', '바닥', '바닥', '바닥', '바닥', '가격', '가격', '가격', '가격', '가격', '주차', '주차', '주차', '주차', '주차', '주차', '주차', '주차', '주차', '주차', '주차', '주차', '주차', '주차', '주차', '주차', '난방', '난방', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '화장실', '화장실', '화장실', '화장실'], ['주차', '주차', '주차', '주차', '주차', '주차', '주차', '가격', '가격', '가격', '가격', '가격', '가격', '가격', '가격', '가격', '가격', '가격', '가격', '가격', '가격', '가격', '가격', '가격', '가격', '가격', '가격', '가격', '가격', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '바닥', '바닥', '바닥', '바닥', '바닥', '바닥', '바닥', '바닥', '바닥', '바닥', '바닥', '바닥', '바닥', '바닥', '바닥', '난방', '난방', '난방', '난방', '난방', '화장실', '화장실', '화장실'], ['시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설', '시설'

In [ ]:
documents = [" ".join(gym) for gym in data_list ]
documents

['바닥 바닥 바닥 바닥 바닥 바닥 바닥 가격 가격 가격 가격 가격 주차 주차 주차 주차 주차 주차 주차 주차 주차 주차 주차 주차 주차 주차 주차 주차 난방 난방 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 화장실 화장실 화장실 화장실',
 '주차 주차 주차 주차 주차 주차 주차 가격 가격 가격 가격 가격 가격 가격 가격 가격 가격 가격 가격 가격 가격 가격 가격 가격 가격 가격 가격 가격 가격 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 바닥 바닥 바닥 바닥 바닥 바닥 바닥 바닥 바닥 바닥 바닥 바닥 바닥 바닥 바닥 난방 난방 난방 난방 난방 화장실 화장실 화장실',
 '시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 바닥 바닥 바닥 바닥 바닥 바닥 바닥 바닥 바닥 난방 난방 난방 주차 주차 주차 주차 주차 주차 주차 주차 가격 가격 가격 화장실 화장실 화장실 화장실 화장실 화장실 화장실 화장실 화장실 화장실 화장실 화장실 화장실',
 '바닥 바닥 바닥 바닥 바닥 바닥 바닥 바닥 바닥 바닥 바닥 바닥 주차 주차 주차 주차 가격 시설 시설 시설 시설 시설 시설 시설 시설 시설 시설 화장실 화장실 난방 난방 난방',
 '주차 주차 주차 주차 주차 바닥 바닥 바닥 바닥 바닥 바닥 바닥 바닥 바닥 바닥 바닥 난방 난방 난방 난방 난방 난방 난방 난방 난방 난방 난방 난방 시설 시설 화장실 가격']

In [ ]:
noun_dict

{'명지전문대학 예체능관 실내체육관': ['바닥', '주차', '가격', '난방', '화장실', '시설'],
 '경기대학교(서울) 본관 실내체육관': ['바닥', '주차', '가격', '난방', '화장실', '시설'],
 '한성대학교 상상관 실내체육관': ['바닥', '주차', '가격', '난방', '화장실', '시설'],
 'N sports 실내체육관': ['바닥', '주차', '가격', '난방', '화장실', '시설'],
 '삼성썬더스 농구클럽 구리다산점 체육관A': ['바닥', '주차', '가격', '난방', '화장실', '시설']}

In [ ]:
# 단어 목록 생성
word_list = ['주차', '바닥', '가격', '난방', '화장실', '시설']

# Create an instance of TfidfVectorizer and fit it on the documents
vectorizer = TfidfVectorizer(vocabulary=word_list)
vectorizer.fit(documents)

# Transform the documents into a document-term matrix
tfidf_matrix = vectorizer.transform(documents)

# Print the resulting document-term matrix with hotel names as row labels
#print(pd.DataFrame(tfidf_matrix.toarray(), columns=word_list, index=gyms))
print(pd.DataFrame(tfidf_matrix.toarray(), columns=word_list, index=gym_list))


                             주차        바닥        가격        난방       화장실  \
명지전문대학 예체능관 실내체육관      0.667246  0.291920  0.208514  0.083406  0.166812   
경기대학교(서울) 본관 실내체육관     0.206150  0.441750  0.647900  0.147250  0.088350   
한성대학교 상상관 실내체육관        0.280056  0.315063  0.105021  0.105021  0.455091   
N sports 실내체육관         0.241649  0.724947  0.060412  0.181237  0.120824   
삼성썬더스 농구클럽 구리다산점 체육관A  0.290619  0.639362  0.058124  0.697486  0.058124   

                             시설  
명지전문대학 예체능관 실내체육관      0.625543  
경기대학교(서울) 본관 실내체육관     0.559550  
한성대학교 상상관 실내체육관        0.770154  
N sports 실내체육관         0.604122  
삼성썬더스 농구클럽 구리다산점 체육관A  0.116248  


In [ ]:
# 체육관별 단어 빈도수 딕셔너리를 데이터프레임으로 변환
df = pd.DataFrame.from_dict(gym_nouns_dict, orient='index')

# 엑셀 파일로 저장
df.to_excel('gym_word_counts.xlsx')

In [ ]:
# 각 체육관과 명사의 상관관계 저장
corr_dict = {}
for gym in gym_nouns_dict.keys():
    corr_dict[gym] = {}
    for noun in gym_nouns_dict[gym].keys():
        corr_dict[gym][noun] = {}
        for other_gym in gym_nouns_dict.keys():
            if other_gym != gym:
                corr_dict[gym][noun][other_gym] = gym_nouns_dict[gym][noun] * gym_nouns_dict[other_gym][noun]

In [ ]:
# 데이터프레임 생성
df = pd.DataFrame.from_dict(gym_nouns_dict, orient='index')

# 전치행렬 적용 (행과 열을 바꿈)
df = df.T

# 표준편차가 0인 열 제거
df = df.loc[:, (df != 0).any(axis=0)]

# 상관계수 계산
corr_matrix = df.corr()

# 결과 출력
print(corr_matrix)

                       명지전문대학 예체능관 실내체육관  경기대학교(서울) 본관 실내체육관  한성대학교 상상관 실내체육관  \
명지전문대학 예체능관 실내체육관               1.000000            0.231613         0.569535   
경기대학교(서울) 본관 실내체육관              0.231613            1.000000         0.150966   
한성대학교 상상관 실내체육관                 0.569535            0.150966         1.000000   
N sports 실내체육관                  0.414155            0.346692         0.545866   
삼성썬더스 농구클럽 구리다산점 체육관A          -0.279225           -0.281129        -0.415524   

                       N sports 실내체육관  삼성썬더스 농구클럽 구리다산점 체육관A  
명지전문대학 예체능관 실내체육관            0.414155              -0.279225  
경기대학교(서울) 본관 실내체육관           0.346692              -0.281129  
한성대학교 상상관 실내체육관              0.545866              -0.415524  
N sports 실내체육관               1.000000               0.354414  
삼성썬더스 농구클럽 구리다산점 체육관A        0.354414               1.000000  


In [ ]:
# pivot 생성

# gym_nouns_dict를 이용해서 데이터프레임 생성
df = pd.DataFrame.from_dict(gym_nouns_dict, orient='index')

# pivot_table 함수를 이용해서 명사를 행으로, 체육관명을 열로 하는 테이블 생성
pivot_df = pd.pivot_table(df.unstack().reset_index(), values=0, index='level_0', columns='level_1')

# 결과 출력
print(pivot_df)

level_1  N sports 실내체육관  경기대학교(서울) 본관 실내체육관  명지전문대학 예체능관 실내체육관  \
level_0                                                          
가격                    1                  22                  5   
난방                    3                   5                  2   
바닥                   12                  15                  7   
시설                   10                  19                 15   
주차                    4                   7                 16   
화장실                   2                   3                  4   

level_1  삼성썬더스 농구클럽 구리다산점 체육관A  한성대학교 상상관 실내체육관  
level_0                                          
가격                           1                3  
난방                          12                3  
바닥                          11                9  
시설                           2               22  
주차                           5                8  
화장실                          1               13  


In [ ]:
corr_df = pivot_df.corr()
print(corr_df)

level_1                N sports 실내체육관  경기대학교(서울) 본관 실내체육관  명지전문대학 예체능관 실내체육관  \
level_1                                                                        
N sports 실내체육관               1.000000            0.346692           0.414155   
경기대학교(서울) 본관 실내체육관           0.346692            1.000000           0.231613   
명지전문대학 예체능관 실내체육관            0.414155            0.231613           1.000000   
삼성썬더스 농구클럽 구리다산점 체육관A        0.354414           -0.281129          -0.279225   
한성대학교 상상관 실내체육관              0.545866            0.150966           0.569535   

level_1                삼성썬더스 농구클럽 구리다산점 체육관A  한성대학교 상상관 실내체육관  
level_1                                                        
N sports 실내체육관                      0.354414         0.545866  
경기대학교(서울) 본관 실내체육관                 -0.281129         0.150966  
명지전문대학 예체능관 실내체육관                  -0.279225         0.569535  
삼성썬더스 농구클럽 구리다산점 체육관A               1.000000        -0.415524  
한성대학교 상상관 실내체육관                    -0.415524         1.